In [106]:
import json
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.layers import Input,Conv1D,MaxPool1D,LSTM,Flatten,Dropout,Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

In [89]:
def cnn_lstm(input_vector, output_size=20, filters=256, strides=3, pool_size=4, units=32, dropout=0.7, lr=0.001)->tf.keras.Model:
    inp = Input((input_vector, 1))
    x = Conv1D( filters, kernel_size=16, strides=strides, activation='relu')(inp)
    x = MaxPool1D(pool_size=pool_size, padding='same')(x)
    x = Conv1D(filters, kernel_size=8, strides=strides, activation='relu', padding='same')(x)
    x = MaxPool1D(pool_size=pool_size, padding='same')(x)
    x = LSTM(units, return_sequences=True, recurrent_activation='hard_sigmoid')(x)
    x = Flatten()(x)
    x = Dropout(dropout)(x)
    preds = Dense(output_size, activation='softmax')(x)
    model = Model(inputs=inp, outputs=preds)
    opt = tf.keras.optimizers.Adam(lr)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])  # optimizer, metrics
    return model

In [90]:
def small_cnn_lstm(input_vector, output_size=20, filters=256, strides=3, pool_size=4, units=32, dropout=0.7, kernel_size=32, lr=0.001):
    inp = Input((input_vector, 1))
    x = Conv1D(filters, kernel_size=kernel_size, strides=strides, activation='relu')(inp)
    x = MaxPool1D(pool_size=pool_size, padding='same')(x)
    x = LSTM(units, return_sequences=True, recurrent_activation='hard_sigmoid')(x)
    x = Flatten()(x)
    x = Dropout(dropout)(x)
    preds = Dense(output_size, activation='softmax')(x)
    model = Model(inputs=inp, outputs=preds)
    opt = tf.keras.optimizers.Adam(lr)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])  # optimizer, metrics
    return model


In [ ]:
cache_occupancy_data='default_small.npz'
sweeps_data='sweeps_small.npz'
sns_data='sns_small.npz'
dns_data='dns_small.npz'

In [137]:
path=sweeps_data

In [138]:
data=np.load(path)
X=np.array( data['X'],dtype=np.float32)
X=X.reshape(*X.shape,-1)
y=np.array( data['y'])

In [135]:
lab_enc=LabelEncoder()
lab_hot=OneHotEncoder()
y_num= lab_enc.fit_transform(y)
y_hot=lab_hot.fit_transform(y_num.reshape(-1,1))
y_hot=y_hot.toarray()

In [150]:
skf = StratifiedShuffleSplit(n_splits=10,random_state=666)
preds=[]
true=[]
for train_index, test_index in skf.split(X, y_num):
    X_train, X_test = X[train_index], X[test_index]
    mean=X_train.mean()
    std=X_train.std()
    X_train_norm=(X_train-mean+1e-10)/std
    X_test_norm=(X_test-mean+1e-10)/std

    y_train, y_test = y_hot[train_index], y_num[test_index]
    length=X_train.shape[1]
    if length<1000:
        model=small_cnn_lstm(X_train.shape[1])
    else:
        model=cnn_lstm(X_train.shape[1])
    model.fit(X_train_norm,y_train,validation_split=0.1,epochs=5,verbose=2,callbacks=[EarlyStopping( patience=5,restore_best_weights=True)])
    preds.extend( model.predict(X_test_norm).argmax(axis=-1))
    true.extend(y_test)
    break
    
    

Train on 1620 samples, validate on 180 samples
Epoch 1/5
1620/1620 - 3s - loss: 2.9108 - accuracy: 0.1105 - val_loss: 2.6085 - val_accuracy: 0.2444
Epoch 2/5
1620/1620 - 2s - loss: 2.3390 - accuracy: 0.2821 - val_loss: 1.6957 - val_accuracy: 0.5222
Epoch 3/5
1620/1620 - 2s - loss: 1.5724 - accuracy: 0.4994 - val_loss: 1.0779 - val_accuracy: 0.6944
Epoch 4/5
1620/1620 - 2s - loss: 1.0661 - accuracy: 0.6660 - val_loss: 0.6877 - val_accuracy: 0.8389
Epoch 5/5
1620/1620 - 2s - loss: 0.7217 - accuracy: 0.7852 - val_loss: 0.4774 - val_accuracy: 0.8833


In [151]:
(preds==true)/200

0.0